# TODO:
- Check which crimes are correl. with socio-economic status (proxy: race (and sex and age?))
- Check whether race/age/sex is correl. with COMPAS scores
- Check whether race/age/sex is correl. with OUR scores
- Try out some other models as well (SVMs, Random Forests, Naive Bayes) 

# Trying stuff out
Import our tools

In [162]:
import pandas as pd

Load in the relevant data, parsing the date fields to a format pandas understands on the way.

In [164]:
df = pd.read_csv(
    'compas-scores.csv',
    usecols=[
        'sex',
        'age',
        #'age_cat',
        'race',
        'juv_fel_count', 
        #'decile_score',
        'juv_misd_count',
        'juv_other_count', 
        'priors_count',
        'days_b_screening_arrest',
        'c_jail_in',
        'c_jail_out',
        'c_offense_date',
        'c_arrest_date', 
        'c_days_from_compas', 
        'c_charge_degree',
        'c_charge_desc', 
        #'is_recid',
        #'num_r_cases',
        #'r_charge_degree', 
        #'r_days_from_arrest', 
        #'r_offense_date',
        #'r_charge_desc', 
        #'r_jail_in', 
        #'r_jail_out', 
        'is_violent_recid',
        #'num_vr_cases', 
        #'vr_charge_degree', 
        #'vr_offense_date',
        #'vr_charge_desc', 
        #'v_decile_score',
        #'v_score_text', 
        #'v_screening_date', 
        #'score_text', 
    ],
    parse_dates=[
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ]
)

In [165]:
print(df.shape)
df = df[df['c_days_from_compas'] <= 50]
print(df.shape)
print(df.columns)
df['c_charge_desc'] = df.c_charge_desc.apply(lambda x: str(x).lower())
print(df.c_charge_desc[['agg' in desc for desc in df.c_charge_desc.tolist()]].unique())

(11757, 16)
(9768, 16)
Index(['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'days_b_screening_arrest',
       'c_jail_in', 'c_jail_out', 'c_offense_date', 'c_arrest_date',
       'c_days_from_compas', 'c_charge_degree', 'c_charge_desc',
       'is_violent_recid'],
      dtype='object')
['aggravated assault w/firearm' 'agg fleeing and eluding'
 'agg battery grt/bod/harm' 'aggravated battery / pregnant'
 'aggravated assault w/dead weap' 'aggrav battery w/deadly weapon'
 'aggrav stalking after injunctn'
 'aggravated battery (firearm/actual possession)'
 'agg fleeing/eluding high speed' 'aggravated battery'
 'aggrav child abuse-agg battery' 'aggravated assault'
 'agg assault w/int com fel dome' 'aggravated assault w/o firearm'
 'aggrav child abuse-causes harm' 'stalking (aggravated)'
 'aggr child abuse-torture,punish' 'aggravated battery on 65/older'
 'aggravated battery (firearm)' 'aggravated manslaughter child'
 'agg abuse elderlly/dis

Prune non-violent out from the dataset

In [188]:
#violent_kw = {'aggravated', 'batt', 'burgl', 'agg', 'violence', 'manslaughter', 'murder', 'bodily',
#                    'armed', 'cruelty', 'crlty', 'assault', 'viol', 'shoot'}
#non_violent_kw = {'w/o violence', 'without violence', 'no bodily harm'}

v_crimes = pd.read_csv('unique_c_charge_desc_lowercase.csv').query('v == "v"').battery.unique()
df = df[df.c_charge_desc.apply(lambda x: x in v_crimes)]
v_crimes.to_csv('unique_violent_crimes.csv')

AttributeError: 'DataFrame' object has no attribute 'c_charge_desc'

In [167]:
#df['c_charge_desc'].value_counts().to_csv('unique_c_charge_desc_lowercase.csv')

Transform all string data to lowercase and remove extra whitespace. This handles some problematic cases where we have e.g. 'Id theft' and 'ID Theft' as separate labels.

In [168]:
df = df.applymap(lambda s : s.lower().strip() if type(s) == str else s)
df['c_charge_desc'].value_counts().to_csv('cadasdas.csv')

Transform all string columns to groups of binary columns. This is probably not the smartest way to go about this, given that we have so many crime labels. The smart thing to do would probably be some type of a crime classification system, e.g. is_violent, is_with_weapon, etc. But that would require a lot of manual work.

In [169]:
df = pd.get_dummies(df)

Check if we have any more object columns (should be none)

In [170]:
df.dtypes.value_counts()

uint8             70
int64              6
datetime64[ns]     4
float64            2
dtype: int64

Calculate jail stay lengths

In [171]:
df['c_days_in_jail'] = (df['c_jail_out'] - df['c_jail_in']).dt.days.fillna(0).astype(int)
#df['r_days_in_jail'] = (df['r_jail_out'] - df['r_jail_in']).dt.days.fillna(0).astype(int)

Drop date columns

In [172]:
df.drop([
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ],
    axis='columns',
    inplace=True
)

Check dtypes again, should be no dates remaining

In [173]:
df.dtypes.value_counts()

uint8      70
int64       7
float64     2
dtype: int64

Still a few floats, see what that is all about

In [174]:
df.select_dtypes(include=['float64'])

,days_b_screening_arrest,c_days_from_compas
0,-1.0,1.0
2,-1.0,1.0
25,-1.0,1.0
29,0.0,1.0
40,0.0,0.0
47,-1.0,1.0
49,-1.0,0.0
58,-1.0,1.0
88,-1.0,1.0
97,0.0,0.0


At least two look like only NaNs, check to be sure

In [175]:
#print(df['num_vr_cases'].value_counts())
#print(df['num_r_cases'].value_counts())

Yeah, NaNs only. We can drop those. The others look pretty meaningless as well, so drop those too.

In [176]:
df.drop([
        #'num_r_cases',
        #'num_vr_cases',
        #'days_b_screening_arrest',
        #'c_days_from_compas',
        #'r_days_from_arrest'

    ],
    axis='columns',
    inplace=True
)
df.dropna(inplace=True)

Check if we have any more NaNs hanging around

In [177]:
for col in df:
    count = len(df[col]) - df[col].count()
    if count:
        print(col, count)

Should be no more NaNs to go, so we can continue to learning.

Split to target and explanatory variables

In [178]:
print(df.columns)
predicted_variable = 'is_violent_recid'
df_ = df.query(predicted_variable + ' in (0, 1)')
print('negative samples: ', len(df) - len(df_))
X = df_.loc[:, df_.columns != predicted_variable]
y = df_.loc[:, df_.columns == predicted_variable]
print(X.shape, y.shape)
n_pos = y[predicted_variable].sum()
print(n_pos / (y.shape[0]))

Index(['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'days_b_screening_arrest', 'c_days_from_compas',
       'is_violent_recid', 'sex_female', 'sex_male', 'race_african-american',
       'race_asian', 'race_caucasian', 'race_hispanic', 'race_native american',
       'race_other', 'c_charge_degree_f', 'c_charge_degree_m',
       'c_charge_desc_agg abuse elderlly/disabled adult',
       'c_charge_desc_agg assault w/int com fel dome',
       'c_charge_desc_agg battery cause bodily',
       'c_charge_desc_agg battery grt/bod/harm',
       'c_charge_desc_agg fleeing and eluding',
       'c_charge_desc_agg fleeing/eluding high speed',
       'c_charge_desc_aggr child abuse-torture,punish',
       'c_charge_desc_aggrav battery w/deadly weapon',
       'c_charge_desc_aggrav child abuse-agg battery',
       'c_charge_desc_aggrav child abuse-causes harm',
       'c_charge_desc_aggravated animal cruelty',
       'c_charge_desc_aggravated assault',
       'c_c

Split to train and test sets

In [179]:
from sklearn import model_selection

seed = 12
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=seed)
pos = y_train.sum()
print(pos, len(y_train) - pos)
pos = y_test.sum()
print(pos, len(y_test) - pos)

is_violent_recid    118
dtype: int64 is_violent_recid    968
dtype: int64
is_violent_recid    10
dtype: int64 is_violent_recid    111
dtype: int64


Train a ridge regression model and check the $R^2$ score. Score of 1 is best possible, values can range to $-\inf$. RidgeCV automatically cross validates (leave-one-out) to figure out the best hyperparameter $\alpha$ 

In [180]:
from sklearn import linear_model, ensemble, neural_network
from sklearn.metrics import precision_recall_fscore_support as pr_f1, accuracy_score as acc, matthews_corrcoef
from time import time
import numpy as np

def print_score(modelname, y, y_hat):
    accuracy = acc(y, y_hat)
    matt = matthews_corrcoef(y.values.ravel(), y_hat)
    p, r, f1, sup = pr_f1(y, y_hat)
    print(modelname)
    print('accuracy:', acc(y, y_hat))
    print('precision:', tuple(p))
    print('recall:', tuple(r))
    print('f1:', tuple(f1))
    print('Matthew:', matt)
    print('')
    
naive_y = np.zeros(len(y_test))
print_score('Naive', y_test, naive_y)

lr = linear_model.LogisticRegression()
lr.fit(X_train, y_train.values.ravel())
print_score('Logreg', y_test, lr.predict(X_test))

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train.values.ravel())
print_score('KNN', y_test, knn.predict(X_test))

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=seed)
rf.fit(X_train, y_train.values.ravel())
print_score('RF', y_test, rf.predict(X_test))

start = time()
from sklearn.ensemble import GradientBoostingClassifier
gbt = GradientBoostingClassifier(learning_rate=2e-3, n_estimators=5000, subsample=.5, random_state=seed)
gbt.fit(X_train, y_train.values.ravel())
print_score('GBTree', y_test, gbt.predict(X_test))
print('gbt trained in {:.2f} seconds'.format(time() - start))

start = time()
mlp = neural_network.MLPClassifier(hidden_layer_sizes=((200,)*2), solver='adam', activation='logistic', random_state=seed)
mlp.fit(X_train, y_train.values.ravel())
print_score('MLP', y_test, mlp.predict(X_test))
print('mlp trained in {:.2f} seconds'.format(time() - start))

start = time()
mlp2 = neural_network.MLPClassifier(hidden_layer_sizes=((200,)*4), solver='adam', activation='logistic', random_state=seed)
mlp2.fit(X_train, y_train.values.ravel())
print_score('MLP2', y_test, mlp2.predict(X_test))
print('mlp2 trained in {:.2f} seconds'.format(time() - start))

start = time()
mlp3 = neural_network.MLPClassifier(hidden_layer_sizes=((128,)*6), solver='adam', activation='logistic', random_state=seed)
mlp3.fit(X_train, y_train.values.ravel())
print_score('MLP3', y_test, mlp3.predict(X_test))
print('mlp3 trained in {:.2f} seconds'.format(time() - start))

Naive
accuracy: 0.9173553719008265
precision: (0.9173553719008265, 0.0)
recall: (1.0, 0.0)
f1: (0.956896551724138, 0.0)
Matthew: 0.0

Logreg
accuracy: 0.9173553719008265
precision: (0.9173553719008265, 0.0)
recall: (1.0, 0.0)
f1: (0.956896551724138, 0.0)
Matthew: 0.0

KNN
accuracy: 0.9090909090909091
precision: (0.9166666666666666, 0.0)
recall: (0.990990990990991, 0.0)
f1: (0.9523809523809523, 0.0)
Matthew: -0.027399831217559546

RF
accuracy: 0.9008264462809917
precision: (0.9159663865546218, 0.0)
recall: (0.9819819819819819, 0.0)
f1: (0.9478260869565216, 0.0)
Matthew: -0.03891168412158467



/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GBTree
accuracy: 0.9090909090909091
precision: (0.9166666666666666, 0.0)
recall: (0.990990990990991, 0.0)
f1: (0.9523809523809523, 0.0)
Matthew: -0.027399831217559546

gbt trained in 14.94 seconds
MLP
accuracy: 0.9008264462809917
precision: (0.9159663865546218, 0.0)
recall: (0.9819819819819819, 0.0)
f1: (0.9478260869565216, 0.0)
Matthew: -0.03891168412158467

mlp trained in 5.66 seconds


/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


MLP2
accuracy: 0.9173553719008265
precision: (0.9173553719008265, 0.0)
recall: (1.0, 0.0)
f1: (0.956896551724138, 0.0)
Matthew: 0.0

mlp2 trained in 2.50 seconds
MLP3
accuracy: 0.9173553719008265
precision: (0.9173553719008265, 0.0)
recall: (1.0, 0.0)
f1: (0.956896551724138, 0.0)
Matthew: 0.0

mlp3 trained in 1.53 seconds


In [187]:
models = (gbt, rf)
y_hat =  np.sum([ml.predict_proba(X_test)[:,1] for ml in models], axis=0) / len(models)
print_score('Comb', y_test, [np.round(y) for y in y_hat])

Comb
accuracy: 0.9173553719008265
precision: (0.9173553719008265, 0.0)
recall: (1.0, 0.0)
f1: (0.956896551724138, 0.0)
Matthew: 0.0



/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [182]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import Callback
from keras.regularizers import l2
import numpy
# fix random seed for reproducibility
numpy.random.seed(seed)


# create model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2()))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model

def evaluate_model():
        y_hat = [np.round(y) for y in model.predict(X_test)]
        print_score('Keras MLP', y_test, y_hat)


class EvalCallback(Callback):
    def __init__(self, interval=20):
        super(Callback, self).__init__()
        self.interval = interval

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.interval == 0:
            evaluate_model()

start = time()
model.fit(X_train, y_train.values.ravel(), epochs=500, batch_size=100, callbacks=[EvalCallback()])
# evaluate the model
print('model trained in {:.2f} second'.format(time() - start))


Epoch 1/500
1086/1086 [==============================] - 1s 816us/step - loss: 0.7258 - acc: 0.8278
Epoch 2/500
1086/1086 [==============================] - 0s 69us/step - loss: 0.4955 - acc: 0.8766
Epoch 3/500
1086/1086 [==============================] - 0s 75us/step - loss: 0.4724 - acc: 0.8840
Epoch 4/500
1086/1086 [==============================] - 0s 64us/step - loss: 0.4399 - acc: 0.8840
Epoch 5/500
1086/1086 [==============================] - 0s 65us/step - loss: 0.4340 - acc: 0.8812
Epoch 6/500
1086/1086 [==============================] - 0s 67us/step - loss: 0.4315 - acc: 0.8840
Epoch 7/500
1086/1086 [==============================] - 0s 64us/step - loss: 0.4065 - acc: 0.8877
Epoch 8/500
1086/1086 [==============================] - 0s 66us/step - loss: 0.4037 - acc: 0.8886
Epoch 9/500
1086/1086 [==============================] - 0s 65us/step - loss: 0.3721 - acc: 0.8858
Epoch 10/500
1086/1086 [==============================] - 0s 72us/step - loss: 0.4087 - acc: 0.8904
Epoch 11

/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1086/1086 [==============================] - 0s 74us/step - loss: 0.3561 - acc: 0.8913
Epoch 24/500
1086/1086 [==============================] - 0s 77us/step - loss: 0.3512 - acc: 0.8913
Epoch 25/500
1086/1086 [==============================] - 0s 68us/step - loss: 0.3676 - acc: 0.8886
Epoch 26/500
1086/1086 [==============================] - 0s 71us/step - loss: 0.3513 - acc: 0.8913
Epoch 27/500
1086/1086 [==============================] - 0s 60us/step - loss: 0.3512 - acc: 0.8895
Epoch 28/500
1086/1086 [==============================] - 0s 61us/step - loss: 0.3330 - acc: 0.8932
Epoch 29/500
1086/1086 [==============================] - 0s 68us/step - loss: 0.3270 - acc: 0.8923
Epoch 30/500
1086/1086 [==============================] - 0s 71us/step - loss: 0.3312 - acc: 0.8904
Epoch 31/500
1086/1086 [==============================] - 0s 65us/step - loss: 0.3476 - acc: 0.8895
Epoch 32/500
1086/1086 [==============================] - 0s 70us/step - loss: 0.3287 - acc: 0.8923
Epoch 33/500


1086/1086 [==============================] - 0s 63us/step - loss: 0.2705 - acc: 0.9015
Keras MLP
accuracy: 0.9173553719008265
precision: (0.9173553719008265, 0.0)
recall: (1.0, 0.0)
f1: (0.956896551724138, 0.0)
Matthew: 0.0

Epoch 101/500
1086/1086 [==============================] - 0s 63us/step - loss: 0.2709 - acc: 0.8978
Epoch 102/500
1086/1086 [==============================] - 0s 62us/step - loss: 0.2725 - acc: 0.8996
Epoch 103/500
1086/1086 [==============================] - 0s 59us/step - loss: 0.2680 - acc: 0.8969
Epoch 104/500
1086/1086 [==============================] - 0s 48us/step - loss: 0.2696 - acc: 0.9033
Epoch 105/500
1086/1086 [==============================] - 0s 63us/step - loss: 0.2758 - acc: 0.8969
Epoch 106/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.2718 - acc: 0.9006
Epoch 107/500
1086/1086 [==============================] - 0s 65us/step - loss: 0.2614 - acc: 0.9015
Epoch 108/500
1086/1086 [==============================] - 0s 62us/s

1086/1086 [==============================] - 0s 55us/step - loss: 0.1902 - acc: 0.9300
Epoch 175/500
1086/1086 [==============================] - 0s 72us/step - loss: 0.1848 - acc: 0.9365
Epoch 176/500
1086/1086 [==============================] - 0s 66us/step - loss: 0.1949 - acc: 0.9328
Epoch 177/500
1086/1086 [==============================] - 0s 61us/step - loss: 0.1775 - acc: 0.9300
Epoch 178/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.1819 - acc: 0.9273
Epoch 179/500
1086/1086 [==============================] - 0s 60us/step - loss: 0.1869 - acc: 0.9300
Epoch 180/500
1086/1086 [==============================] - 0s 59us/step - loss: 0.1737 - acc: 0.9328
Keras MLP
accuracy: 0.8842975206611571
precision: (0.9145299145299145, 0.0)
recall: (0.963963963963964, 0.0)
f1: (0.9385964912280702, 0.0)
Matthew: -0.055497775450941096

Epoch 181/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.1723 - acc: 0.9346
Epoch 182/500
1086/1086 [==========

1086/1086 [==============================] - 0s 62us/step - loss: 0.1195 - acc: 0.9549
Epoch 248/500
1086/1086 [==============================] - 0s 59us/step - loss: 0.1139 - acc: 0.9595
Epoch 249/500
1086/1086 [==============================] - 0s 57us/step - loss: 0.1182 - acc: 0.9567
Epoch 250/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.1071 - acc: 0.9632
Epoch 251/500
1086/1086 [==============================] - 0s 63us/step - loss: 0.1139 - acc: 0.9576
Epoch 252/500
1086/1086 [==============================] - 0s 60us/step - loss: 0.1077 - acc: 0.9586
Epoch 253/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.1185 - acc: 0.9549
Epoch 254/500
1086/1086 [==============================] - 0s 60us/step - loss: 0.1015 - acc: 0.9595
Epoch 255/500
1086/1086 [==============================] - 0s 65us/step - loss: 0.1107 - acc: 0.9613
Epoch 256/500
1086/1086 [==============================] - 0s 62us/step - loss: 0.1164 - acc: 0.9604
Epoc

1086/1086 [==============================] - 0s 69us/step - loss: 0.0789 - acc: 0.9659
Epoch 322/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.0862 - acc: 0.9641
Epoch 323/500
1086/1086 [==============================] - 0s 50us/step - loss: 0.0820 - acc: 0.9696
Epoch 324/500
1086/1086 [==============================] - 0s 51us/step - loss: 0.0839 - acc: 0.9650
Epoch 325/500
1086/1086 [==============================] - 0s 66us/step - loss: 0.0848 - acc: 0.9733
Epoch 326/500
1086/1086 [==============================] - 0s 75us/step - loss: 0.0746 - acc: 0.9724
Epoch 327/500
1086/1086 [==============================] - 0s 63us/step - loss: 0.0775 - acc: 0.9687
Epoch 328/500
1086/1086 [==============================] - 0s 62us/step - loss: 0.0794 - acc: 0.9687
Epoch 329/500
1086/1086 [==============================] - 0s 56us/step - loss: 0.0794 - acc: 0.9724
Epoch 330/500
1086/1086 [==============================] - 0s 60us/step - loss: 0.0818 - acc: 0.9696
Epoc

1086/1086 [==============================] - 0s 50us/step - loss: 0.0744 - acc: 0.9705
Epoch 397/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.0712 - acc: 0.9724
Epoch 398/500
1086/1086 [==============================] - 0s 64us/step - loss: 0.0691 - acc: 0.9761
Epoch 399/500
1086/1086 [==============================] - 0s 57us/step - loss: 0.0665 - acc: 0.9742
Epoch 400/500
1086/1086 [==============================] - 0s 65us/step - loss: 0.0596 - acc: 0.9797
Keras MLP
accuracy: 0.8677685950413223
precision: (0.9279279279279279, 0.2)
recall: (0.9279279279279279, 0.2)
f1: (0.9279279279279279, 0.20000000000000004)
Matthew: 0.12792792792792793

Epoch 401/500
1086/1086 [==============================] - 0s 87us/step - loss: 0.0531 - acc: 0.9807
Epoch 402/500
1086/1086 [==============================] - 0s 78us/step - loss: 0.0672 - acc: 0.9770
Epoch 403/500
1086/1086 [==============================] - 0s 64us/step - loss: 0.0623 - acc: 0.9779
Epoch 404/500
1086/1

1086/1086 [==============================] - 0s 62us/step - loss: 0.0524 - acc: 0.9816
Epoch 470/500
1086/1086 [==============================] - 0s 55us/step - loss: 0.0593 - acc: 0.9807
Epoch 471/500
1086/1086 [==============================] - 0s 53us/step - loss: 0.0547 - acc: 0.9834
Epoch 472/500
1086/1086 [==============================] - 0s 58us/step - loss: 0.0535 - acc: 0.9843
Epoch 473/500
1086/1086 [==============================] - 0s 62us/step - loss: 0.0556 - acc: 0.9770
Epoch 474/500
1086/1086 [==============================] - 0s 61us/step - loss: 0.0569 - acc: 0.9816
Epoch 475/500
1086/1086 [==============================] - 0s 63us/step - loss: 0.0431 - acc: 0.9853
Epoch 476/500
1086/1086 [==============================] - 0s 69us/step - loss: 0.0504 - acc: 0.9807
Epoch 477/500
1086/1086 [==============================] - 0s 81us/step - loss: 0.0544 - acc: 0.9807
Epoch 478/500
1086/1086 [==============================] - 0s 80us/step - loss: 0.0452 - acc: 0.9843
Epoc

In [183]:

print_score('Keras MLP', y_test, model.predict_classes(X_test))

Keras MLP
accuracy: 0.8429752066115702
precision: (0.9259259259259259, 0.15384615384615385)
recall: (0.9009009009009009, 0.2)
f1: (0.91324200913242, 0.17391304347826086)
Matthew: 0.08971663566887349



In [184]:
c = model.predict(X_test)
res = model.predict_proba(X_test)
[(r[1], c) for r, c in zip(res, c)]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
!jt -r 

That's... not horrible. Let's check which features have the greatest effect on the thing. 

In [ ]:
#coeffs = [(coef, col) for col, coef in zip(X.columns, model.coef_[0])]

import numpy as np
coefs = [np.mean(c) for c in model.coefs_[0]]
coeffs = [(coef, col) for col, coef in zip(X.columns, coefs)] # for neural net
coeffs = sorted(coeffs, reverse=True)
for a, b in coeffs: print(a, b)


In [ ]:
#coeffs = [(coef, col) for col, coef in zip(X.columns, model.coef_[0])]
#coeffs = sorted(coeffs, reverse=True)

for a, b in coeffs[:20]: print(a, b)
    
print('...')

orig = {col:coef for coef, col in coeffs}
abss = sorted([(abs(coef), col) for coef, col in coeffs])
near_zero = sorted([(orig[col], col) for coef, col in abss[:50]], reverse=True)
for a, b in near_zero: print(a, b)


print('...')

for a, b in coeffs[-20:]: print(a, b)

Let's check the non-crime label ones specifically

In [ ]:
for a, b in coeffs: print(a, b) if 'charge_desc' not in b else ...

Try linear support vector regression next

In [ ]:
from sklearn import svm

#model = svm.LinearSVR(epsilon=0.5)
#scores = model_selection.cross_val_score(model, X, y.values.ravel(), cv=10, n_jobs=2)
#scores

Marginally better, still pretty bad. Let's try support vector regression with an RBF. In principle it has an infinite-dimensional faeture space so that should get to something reasonable at the cost of possibly overfitting.

In [ ]:
from sklearn import svm

#model = svm.SVR(epsilon=0.5)
#scores = model_selection.cross_val_score(model, X, y.values.ravel(), cv=10, n_jobs=3)
#scores

It's... not bad(?) considering we are using $\frac{1}{3}$ of the data for testing on each round of the cross validation. The resulting $R^2$ values indicate $R \approx 0.6$ which is by no means an abysmal degree of correlation. At the same time, we would expect a significantly higher degree of success based on what the task actually **is**.

Might be problem with the nature of the data, more specifically not enough data for practical learning considering the size of the (unmodified) feature space. We only have ~10x data points to the number of features.

Might want to try, say, a random forest or alternatively doing some tricks with sample generation. Might also be issues regarding colinearity, but that's a bit beyond me, to be quite hones.